In [30]:
def filter_elos(sample):
    #sample = sample.drop(['Player_White','Player_Black','Tournament','City'],axis = 1) # this is temporary because 

    # count number of row with unr in elo 
    initial = len(sample)
    sample = sample[sample['ELO_Black'].apply(lambda x: str(x).isdigit())]
    sample = sample[sample['ELO_White'].apply(lambda x: str(x).isdigit())]
    resulting = len(sample)
    
    print(initial-resulting , ' rows dropped')
    return sample

def filter_tournament(sample):
    initial = len(sample)
    Under = sample['Tournament'].apply(lambda x: ('U' in str(x)))
    Women = sample['Tournament'].apply(lambda x: ('women' in str(x).lower()))
    Men = sample['Tournament'].apply(lambda x: ('men' in str(x).lower()))
    
    sample.loc[:,'Tournament'] = 0 # General
    sample.loc[Under,'Tournament'] = 1
    sample.loc[Men,'Tournament'] = 2
    sample.loc[Women,'Tournament'] = 3
                                      
    resulting = len(sample)
    print(initial-resulting , ' rows dropped')
    return sample

def name_hash(name):
    name = name.lower()
    val = 0
    mod = 1000000007
    
    for i in name:
        val = val * 30 + ord(i) - ord('a')
        val = val % mod
        
    return val


def filter_player_names(sample):
    
    for i, row in sample.iterrows():
        sample.set_value(i,'Player_White',name_hash(sample.loc[i]['Player_White']))
        sample.set_value(i,'Player_Black',name_hash(sample.loc[i]['Player_Black']))

    return sample

In [43]:
import pandas as pd

def readTestInput():
    df = pd.read_csv("sujit.csv")
    #print(df)
    df = filter_player_names(df)
    df = filter_tournament(df)
    df = filter_elos(df)
    print("----------------Modified------------------")
    print(df)
    modified_df = pd.concat([df.loc[:,'Tournament'], df.loc[:, 'Year':'Eval_Diff']], axis=1)
    print(modified_df)


0  rows dropped
0  rows dropped
----------------Modified------------------
   Tournament     City  Year Player_White Player_Black  ELO_White  ELO_Black  \
0           3     Bled  2002    690417527    570336576       2247       2241   
1           0  Yerevan  2012    496421636    199480117       2583       2640   
2           1   Prague  2012    951471481    341321752       1777       1766   
3           0  Germany  2007    328103520    354637944       2235       2227   
4           0   Zenica  1989    918457350    897301923       2490       2510   
5           0  Algarve  1995    674084283    365772728       2245       2245   
6           1  Oberhof  2011    942501432    600011538       1749       1743   
7           0  Wolvega  2012    812153516    216162803       2593       2641   

   Turn  Move#  QueenWhite   ...    RookWhite  RookBlack  BishopWhite  \
0    -1     27           1   ...            2          2            1   
1     1     21           1   ...            2          1  

/home/edexworld/.local/lib/python3.5/site-packages/ipykernel_launcher.py:44: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
/home/edexworld/.local/lib/python3.5/site-packages/ipykernel_launcher.py:45: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead


In [44]:
import pickle
rfc = pickle.load(open('rfcModelMini.sav', 'rb'))

In [46]:
predicted = rfc.predict(modified_df)
print(predicted)

[ 1  0 -1 -1  0  1  1  0]


In [47]:
import pickle
reg = pickle.load(open('rfrModelMini.sav', 'rb'))

In [49]:
predicted = reg.predict(modified_df)
print(predicted)

[ 0.23804056  0.01084988 -0.81685703 -0.37747     0.05939496  0.52767534
  0.54310126  0.06686336]


In [50]:
flask import Flask, jsonify, request
app = Flask(__name__)

languages = [{'name':'JavaScript'}, {'name':'Python'}, {'name':'Ruby'}]

predictions = {}

@app.route('/', methods=['GET'])
def hello_world():
        return jsonify({'message':'It works!'})

@app.route('/lang', methods=['GET'])
def returnAll():
        return jsonify({'languages':languages})

@app.route('/test', methods=['GET'])
def test():
        param = request.args.get('param')
        result = predicted = reg.predict(modified_df)
        for i in range(len(result)):
            predictions[i] = result[i]
        print(predictions)
        return jsonify({'predictions':predictions})

if __name__ == '__main__':
        app.run(debug=True, port=8080)


SyntaxError: invalid syntax (<ipython-input-50-6764e2908cd9>, line 1)